In [1]:
import torch
torch.set_num_threads(16)
torch.set_num_interop_threads(16)

In [2]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF, BPR, SANSA, BiVAECF, LightGCN, RecVAE, EASE, NGCF, VAECF, IBPR, NeuMF, HPF
from cornac.metrics import Precision, Recall, NDCG, MAP, MRR

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### 1. Import datasets

In [3]:
def stratified_ranking_split(
    df: pd.DataFrame,
    entity_field: str,
    test_size: float = 0.1,
    random_state: int | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Splits a ranking-based dataset into training and test sets while preserving the entity distribution.

    This function is useful for ranking models (e.g., next-best-offer, personalized recommendations) where each
    entity (e.g., user) has multiple interactions with different items, and stratification ensures that
    different user interaction levels are maintained in both splits.

    Parameters:
        df: The ranking dataset, where each row represents an interaction between an entity (e.g., user)
            and an item (e.g., game, offer).
        entity_field: The column representing the entity to be stratified.
        test_size: Fraction of unique entities to allocate to the test set.
        random_state: Random seed for reproducibility.

    Returns:
        DataFrames containing training and test data.

    Example:
        >>> train_validation_df, test_df = stratified_ranking_split(df, entity_field='user_id', test_size=0.1)
        >>> train_df, validation_df = stratified_ranking_split(
        ...     train_validation_df, entity_field='user_id', test_size=0.1
        ... )
        >>> print(train_df.shape, validation_df.shape, test_df.shape)
    """
    entity_interaction_counts = df[entity_field].value_counts()

    interaction_frequencies = entity_interaction_counts.value_counts()
    stratifiable_interaction_counts = interaction_frequencies[interaction_frequencies >= 2].index

    stratifiable_entities = entity_interaction_counts[
        entity_interaction_counts.isin(stratifiable_interaction_counts)
    ].index
    non_stratifiable_entities = entity_interaction_counts[
        ~entity_interaction_counts.isin(stratifiable_interaction_counts)
    ].index

    train_strat, test_strat = (
        train_test_split(
            stratifiable_entities,
            test_size=test_size,
            stratify=entity_interaction_counts[stratifiable_entities],
            random_state=random_state,
        )
        if len(stratifiable_entities) > 1
        else (stratifiable_entities, [])
    )

    if len(non_stratifiable_entities) > 1:
        train_non_strat, test_non_strat = train_test_split(
            non_stratifiable_entities,
            test_size=test_size,
            random_state=random_state,
        )
    else:
        train_non_strat = non_stratifiable_entities
        test_non_strat = []

    train_users = np.concatenate([train_strat, train_non_strat])
    test_users = np.concatenate([test_strat, test_non_strat])

    return df[df[entity_field].isin(train_users)], df[df[entity_field].isin(test_users)]

In [4]:
SEED = 123

metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), MAP(), MRR()]

In [5]:
steam_dataset = (
    pd.read_csv(
        "/Users/a-shyraliev/phd/rec-sys-research/collab_filtering_battlefield/steam_recommendations.csv",
        usecols=['user_id', 'app_id', 'hours'],
    )
    .loc[:, ['user_id', 'app_id', 'hours']]
    .drop_duplicates()
)

In [6]:
steam_dataset['user_id'].nunique(), steam_dataset['app_id'].nunique()

(13781059, 37610)

In [7]:
_, steam_dataset_sample_str = stratified_ranking_split(
    steam_dataset,
    entity_field='user_id',
    test_size=0.01,
    random_state=SEED,
)

del steam_dataset, _

In [8]:
rs = RatioSplit(data=steam_dataset_sample_str.values, test_size=0.2, rating_threshold=0.0, seed=SEED)
rs.train_set.csr_matrix, rs.test_set.csr_matrix

(<121521x14807 sparse matrix of type '<class 'numpy.float64'>'
 	with 325828 stored elements in Compressed Sparse Row format>,
 <121521x14807 sparse matrix of type '<class 'numpy.float64'>'
 	with 62707 stored elements in Compressed Sparse Row format>)

### 1. MF hyperparams optimization

In [25]:
mf_models = [
    MF(name='MF k=10, lambda_reg=0.02', k=10, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
    MF(name='MF k=50, lambda_reg=0.02', k=50, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
    MF(name='MF k=100, lambda_reg=0.02', k=100, max_iter=200, lambda_reg=0.02, use_bias=True, seed=SEED),
    MF(name='MF k=10, lambda_reg=0.1', k=10, max_iter=200, lambda_reg=0.1, use_bias=True, seed=SEED),
    MF(name='MF k=50, lambda_reg=0.1', k=50, max_iter=200, lambda_reg=0.1, use_bias=True, seed=SEED),
    MF(name='MF k=100, lambda_reg=0.1', k=100, max_iter=200, lambda_reg=0.1, use_bias=True, seed=SEED),
    MF(name='MF k=10, lambda_reg=1', k=10, max_iter=200, lambda_reg=1, use_bias=True, seed=SEED),
    MF(name='MF k=50, lambda_reg=1', k=50, max_iter=200, lambda_reg=1, use_bias=True, seed=SEED),
    MF(name='MF k=100, lambda_reg=1', k=100, max_iter=200, lambda_reg=1, use_bias=True, seed=SEED),
]

In [ ]:
cornac.Experiment(
    eval_method=rs,
    models=mf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                          | MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------- + --- + ------- + ------------ + --------- + --------- + --------
MF k=10, lambda_reg=0.02  | nan |  0.0001 |       0.0000 |    0.0003 |    0.6121 |  21.6584
MF k=50, lambda_reg=0.02  | nan |  0.0001 |       0.0000 |    0.0003 |    1.4565 |  31.7773
MF k=100, lambda_reg=0.02 | nan |  0.0001 |       0.0000 |    0.0003 |    4.6947 |  30.5076
MF k=10, lambda_reg=0.1   | nan |  0.0001 |       0.0000 |    0.0003 |    0.5838 |  21.4168
MF k=50, lambda_reg=0.1   | nan |  0.0001 |       0.0000 |    0.0003 |    1.4895 |  32.6081
MF k=100, lambda_reg=0.1  | nan |  0.0001 |       0.0000 |    0.0003 |    5.0005 |  32.5445
MF k=10, lambda_reg=1     | nan |  0.0001 |       0.0000 |    0.0003 |    0.6227 |  21.9549
MF k=50, lambda_reg=1     | nan |  0.0001 |       0.0000 |    0.0003 |    1.6991 |  32.9640
MF k=100, lambda_reg=1    | nan |  0.0001 |       0.0000 |    0.0003 

### 2. PMF hyperparams optimization

In [27]:
pmf_models = [
    PMF(name='PMF k=5, lambda_reg=0.001', k=5, max_iter=200, lambda_reg=0.001, seed=SEED),
    PMF(name='PMF k=10, lambda_reg=0.001', k=10, max_iter=200, lambda_reg=0.001, seed=SEED),
    PMF(name='PMF k=15, lambda_reg=0.001', k=15, max_iter=200, lambda_reg=0.001, seed=SEED),
    PMF(name='PMF k=5, lambda_reg=0.1', k=5, max_iter=200, lambda_reg=0.1, seed=SEED),
    PMF(name='PMF k=10, lambda_reg=0.1', k=10, max_iter=200, lambda_reg=0.1, seed=SEED),
    PMF(name='PMF k=15, lambda_reg=0.1', k=15, max_iter=200, lambda_reg=0.1, seed=SEED),
    PMF(name='PMF k=5, lambda_reg=1', k=5, max_iter=200, lambda_reg=1, seed=SEED),
    PMF(name='PMF k=10, lambda_reg=1', k=10, max_iter=200, lambda_reg=1, seed=SEED),
    PMF(name='PMF k=15, lambda_reg=1', k=15, max_iter=200, lambda_reg=1, seed=SEED),
]

In [28]:
cornac.Experiment(
    eval_method=rs,
    models=pmf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                           |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
PMF k=5, lambda_reg=0.001  | 0.0019 |  0.0019 |       0.0005 |    0.0039 |    5.5292 |  44.1669
PMF k=10, lambda_reg=0.001 | 0.0017 |  0.0017 |       0.0005 |    0.0036 |    9.4754 |  47.0882
PMF k=15, lambda_reg=0.001 | 0.0008 |  0.0005 |       0.0001 |    0.0011 |   14.2352 |  45.9387
PMF k=5, lambda_reg=0.1    | 0.0007 |  0.0000 |       0.0000 |    0.0000 |    5.2613 |  45.3645
PMF k=10, lambda_reg=0.1   | 0.0007 |  0.0000 |       0.0000 |    0.0000 |   10.0980 |  47.8467
PMF k=15, lambda_reg=0.1   | 0.0007 |  0.0000 |       0.0000 |    0.0000 |   14.9663 |  47.6700
PMF k=5, lambda_reg=1      | 0.0006 |  0.0002 |       0.0001 |    0.0005 |    5.5415 |  45.5808
PMF k=10, lambda_reg=1     | 0.0008 |  0.0004 |       0.0001 |    0.0006 |   10.5913 |  47.4731
PMF k=15, lambda_reg=1     | 

### 3. BPR hyperparams optimization

In [29]:
bpr_models = [
    BPR(name='BPR k=10, lambda_reg=0.01', k=10, max_iter=200, lambda_reg=0.01, seed=SEED),
    BPR(name='BPR k=50, lambda_reg=0.01', k=50, max_iter=200, lambda_reg=0.01, seed=SEED),
    BPR(name='BPR k=100, lambda_reg=0.01', k=100, max_iter=200, lambda_reg=0.01, seed=SEED),
    BPR(name='BPR k=10, lambda_reg=0.1', k=10, max_iter=200, lambda_reg=0.1, seed=SEED),
    BPR(name='BPR k=50, lambda_reg=0.1', k=50, max_iter=200, lambda_reg=0.1, seed=SEED),
    BPR(name='BPR k=100, lambda_reg=0.1', k=100, max_iter=200, lambda_reg=0.1, seed=SEED),
    BPR(name='BPR k=10, lambda_reg=1', k=10, max_iter=200, lambda_reg=1, seed=SEED),
    BPR(name='BPR k=50, lambda_reg=1', k=50, max_iter=200, lambda_reg=1, seed=SEED),
    BPR(name='BPR k=100, lambda_reg=1', k=100, max_iter=200, lambda_reg=1, seed=SEED),
]

In [30]:
cornac.Experiment(
    eval_method=rs,
    models=bpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                           |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
BPR k=10, lambda_reg=0.01  | 0.0308 |  0.0312 |       0.0092 |    0.0561 |    3.9516 |  45.1212
BPR k=50, lambda_reg=0.01  | 0.0309 |  0.0316 |       0.0094 |    0.0575 |    8.2983 |  57.4187
BPR k=100, lambda_reg=0.01 | 0.0307 |  0.0313 |       0.0093 |    0.0566 |   11.7384 |  57.0230
BPR k=10, lambda_reg=0.1   | 0.0307 |  0.0310 |       0.0090 |    0.0554 |    4.2672 |  47.3254
BPR k=50, lambda_reg=0.1   | 0.0303 |  0.0308 |       0.0091 |    0.0565 |    8.4085 |  57.3794
BPR k=100, lambda_reg=0.1  | 0.0281 |  0.0283 |       0.0087 |    0.0544 |   12.1549 |  55.3509
BPR k=10, lambda_reg=1     | 0.0300 |  0.0307 |       0.0092 |    0.0560 |    3.9472 |  44.8775
BPR k=50, lambda_reg=1     | 0.0299 |  0.0289 |       0.0081 |    0.0509 |    7.5961 |  52.7927
BPR k=100, lambda_reg=1    | 

### 4. BiVAECF hyperparams optimization

In [32]:
bivaecf_models = [
    BiVAECF(name='BiVAECF k=10, encoder_structure=[20]', k=10, encoder_structure=[20], n_epochs=10, use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=20, encoder_structure=[20]', k=50, encoder_structure=[20], n_epochs=10, use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=30, encoder_structure=[20]', k=100, encoder_structure=[20], n_epochs=10, use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=10, encoder_structure=[40]', k=10, encoder_structure=[40], n_epochs=10, use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=50, encoder_structure=[40]', k=50, encoder_structure=[40], n_epochs=10, use_gpu=False, seed=SEED),
    BiVAECF(name='BiVAECF k=100, encoder_structure=[40]', k=100, encoder_structure=[40], n_epochs=10, use_gpu=False, seed=SEED),
]

In [33]:
cornac.Experiment(
    eval_method=rs,
    models=bivaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                                      |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
BiVAECF k=10, encoder_structure=[20]  | 0.0167 |  0.0151 |       0.0044 |    0.0279 |  379.0551 |  47.1337
BiVAECF k=20, encoder_structure=[20]  | 0.0122 |  0.0094 |       0.0029 |    0.0183 |  350.1261 |  47.3764
BiVAECF k=30, encoder_structure=[20]  | 0.0053 |  0.0049 |       0.0020 |    0.0106 |  363.0580 |  47.1756
BiVAECF k=10, encoder_structure=[40]  | 0.0126 |  0.0131 |       0.0035 |    0.0221 |  355.6089 |  44.8915
BiVAECF k=50, encoder_structure=[40]  | 0.0091 |  0.0090 |       0.0035 |    0.0186 |  359.7766 |  47.6155
BiVAECF k=100, encoder_structure=[40] | 0.0076 |  0.0083 |       0.0029 |    0.0145 |  371.4512 |  46.8471



### 5. RecVAE hyperparams optimization

In [43]:
recvae_models = [
    RecVAE(name='RecVAE hidden_dim=600, latent_dim=200', hidden_dim=600, latent_dim=200, n_epochs=10, use_gpu=False, seed=SEED),
]

In [44]:
cornac.Experiment(
    eval_method=rs,
    models=recvae_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                                      |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
RecVAE hidden_dim=600, latent_dim=200 | 0.0304 | 0.0463 |  0.0339 |       0.0107 |    0.0585 | 4152.2937 | 118.4998



### 6. EASE hyperparams optimization

In [19]:
ease_models = [
    EASE(name='EASE lamb=500, posB=True', lamb=500, posB=True, seed=SEED),
    EASE(name='EASE lamb=300, posB=True', lamb=300, posB=True, seed=SEED),
    EASE(name='EASE lamb=800, posB=True', lamb=800, posB=True, seed=SEED),
    EASE(name='EASE lamb=500, posB=False', lamb=500, posB=False, seed=SEED),
    EASE(name='EASE lamb=300, posB=False', lamb=300, posB=False, seed=SEED),
    EASE(name='EASE lamb=800, posB=False', lamb=800, posB=False, seed=SEED),
]

In [20]:
cornac.Experiment(
    eval_method=rs,
    models=ease_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                          |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
EASE lamb=500, posB=True  | 0.0406 |  0.0460 |       0.0126 |    0.0771 |   48.7037 |  40.0161
EASE lamb=300, posB=True  | 0.0404 |  0.0456 |       0.0125 |    0.0767 |   46.7271 |  38.7585
EASE lamb=800, posB=True  | 0.0408 |  0.0461 |       0.0126 |    0.0771 |   48.3324 |  40.2958
EASE lamb=500, posB=False | 0.0394 |  0.0453 |       0.0123 |    0.0759 |   48.9436 |  44.1733
EASE lamb=300, posB=False | 0.0392 |  0.0450 |       0.0122 |    0.0756 |   48.5401 |  44.9813
EASE lamb=800, posB=False | 0.0396 |  0.0455 |       0.0124 |    0.0761 |   48.5227 |  43.8767



### 7. HPF hyperparams optimization

In [21]:
hpf_models = [
    HPF(name='HPF k=5, hierarchical=True', k=5, hierarchical=True, seed=SEED),
    HPF(name='HPF k=10, hierarchical=True', k=10, hierarchical=True, seed=SEED),
    HPF(name='HPF k=15, hierarchical=True', k=15, hierarchical=True, seed=SEED),
    HPF(name='HPF k=5, hierarchical=False', k=5, hierarchical=False, seed=SEED),
    HPF(name='HPF k=10, hierarchical=False', k=10, hierarchical=False, seed=SEED),
    HPF(name='HPF k=15, hierarchical=False', k=15, hierarchical=False, seed=SEED),
]

In [22]:
cornac.Experiment(
    eval_method=rs,
    models=hpf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

Learning...
Learning completed!
Learning...
Learning completed!
Learning...
Learning completed!
Learning...
Learning completed!
Learning...
Learning completed!
Learning...
Learning completed!

TEST:
...
                             |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
---------------------------- + ------ + ------- + ------------ + --------- + --------- + --------
HPF k=5, hierarchical=True   | 0.0193 |  0.0205 |       0.0060 |    0.0369 |   52.4575 |  43.8802
HPF k=10, hierarchical=True  | 0.0175 |  0.0191 |       0.0057 |    0.0345 |  103.9741 |  44.7900
HPF k=15, hierarchical=True  | 0.0157 |  0.0167 |       0.0051 |    0.0300 |  160.5110 |  45.2499
HPF k=5, hierarchical=False  | 0.0198 |  0.0214 |       0.0065 |    0.0389 |   51.9678 |  43.6735
HPF k=10, hierarchical=False | 0.0187 |  0.0195 |       0.0056 |    0.0336 |   99.7340 |  43.7865
HPF k=15, hierarchical=False | 0.0162 |  0.0164 |       0.0049 |    0.0279 |  157.7568 |  43.2988



### 8. IBPR hyperparams optimization

In [49]:
ibpr_models = [
    IBPR(name='IBPR k=20, lamda=0.001', k=20, lamda=0.001),
]

In [ ]:
cornac.Experiment(
    eval_method=rs,
    models=ibpr_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

In [ ]:
## too long to train
472m of training time - still not converged

### 9. SANSA hyperparams optimization

In [45]:
sansa_models = [
    # SANSA(name='SANSA l2=1, factorizer_class=CHOLMOD', l2=1, factorizer_class="CHOLMOD", verbose=False, seed=SEED),
    # SANSA(name='SANSA l2=10, factorizer_class=CHOLMOD', l2=10, factorizer_class="CHOLMOD", verbose=False, seed=SEED),
    # SANSA(name='SANSA l2=500, factorizer_class=CHOLMOD', l2=500, factorizer_class="CHOLMOD", verbose=False, seed=SEED),
    SANSA(name='SANSA l2=1, factorizer_class=ICF', l2=1, factorizer_class="ICF", verbose=False, seed=SEED),
    SANSA(name='SANSA l2=10, factorizer_class=ICF', l2=10, factorizer_class="ICF", verbose=False, seed=SEED),
    SANSA(name='SANSA l2=500, factorizer_class=ICF', l2=500, factorizer_class="ICF", verbose=False, seed=SEED),
]

In [46]:
cornac.Experiment(
    eval_method=rs,
    models=sansa_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()

INFO:sansa.core.factorizers:Computing incomplete Cholesky decomposition of X^TX + 1*I...
INFO:sansa.core.factorizers:Finding a fill-in reducing ordering (method = colamd)...
INFO:sansa.core.factorizers:Computing X^TX...
INFO:sansa.core.factorizers:
                X^TX info:
                    shape = (14807, 14807) 
                    nnz = 2905784 
                    density = 1.325346% 
                    size = 23.3 MB
                
INFO:sansa.core.factorizers:Sorting indices of A...
INFO:sansa.core.factorizers:Casting indptr of A to int64...
INFO:sansa.core.factorizers:Casting indices of A to int64...
INFO:sansa.core.factorizers:Computing approximate Cholesky decomposition (method = ICF)...
INFO:sansa.core._ops._factor_ops:
                Incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A 
                with L2=1.0000e+00. Continuing with L2=1.0010e+00.
                
INFO:sansa.core._ops._factor_ops:
                Incomple


TEST:
...
                                   |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
---------------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
SANSA l2=1, factorizer_class=ICF   | 0.0191 | 0.0300 |  0.0225 |       0.0066 |    0.0375 |   16.8820 |  36.4241
SANSA l2=10, factorizer_class=ICF  | 0.0191 | 0.0300 |  0.0225 |       0.0066 |    0.0375 |   16.0139 |  35.5143
SANSA l2=500, factorizer_class=ICF | 0.0191 | 0.0300 |  0.0226 |       0.0066 |    0.0376 |    6.6424 |  33.9904



### 10. VAECF hyperparams optimization

In [47]:
vaecf_models = [
    VAECF(name='VAECF k=5, likelihood=mult', k=5, likelihood="mult", n_epochs=10, use_gpu=False, seed=SEED),
    VAECF(name='VAECF k=5, likelihood=gaus', k=5, likelihood="gaus", n_epochs=10, use_gpu=False, seed=SEED),
    VAECF(name='VAECF k=10, likelihood=mult', k=10, likelihood="mult", n_epochs=10, use_gpu=False, seed=SEED),
    VAECF(name='VAECF k=10, likelihood=gaus', k=10, likelihood="gaus", n_epochs=10, use_gpu=False, seed=SEED),
]

In [48]:
cornac.Experiment(
    eval_method=rs,
    models=vaecf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
                            |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) |  Test (s)
--------------------------- + ------ + ------ + ------- + ------------ + --------- + --------- + ---------
VAECF k=5, likelihood=mult  | 0.0298 | 0.0410 |  0.0316 |       0.0100 |    0.0614 |  297.3855 |   58.0917
VAECF k=5, likelihood=gaus  | 0.0268 | 0.0354 |  0.0263 |       0.0079 |    0.0511 | 3190.6507 | 1090.2030
VAECF k=10, likelihood=mult | 0.0290 | 0.0396 |  0.0300 |       0.0095 |    0.0582 | 2086.4787 |   57.7111
VAECF k=10, likelihood=gaus | 0.0269 | 0.0356 |  0.0264 |       0.0079 |    0.0511 | 1260.1899 |   56.8296



### 11. NeuMF hyperparams optimization

In [9]:
neumf_models = [
    NeuMF(verbose=False, backend="pytorch", seed=SEED),
]

In [10]:
cornac.Experiment(
    eval_method=rs,
    models=neumf_models,
    metrics=metrics,
    user_based=True,
    save_dir=None,
).run()


TEST:
...
      |    MAP |    MRR | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
----- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
NeuMF | 0.0263 | 0.0381 |  0.0275 |       0.0087 |    0.0491 |  276.8628 | 124.8260

